# 添加 brunel 包

In [ ]:
%Addjar --magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar -f

# 1 客户流失率可视化分析
## 1.1导入客户流失率数据

In [ ]:
//此部分需要从Object Storage中读取数据
//1.添加CUST_Rate.csv到本工程的Object Storage
//2.在notebook右上角点击数据窗口，再点击下拉菜单的Insert Spark SQL DataFrame
//3.运行输出数据dfData1

In [2]:
// @hidden_cell
// This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
// You might want to remove those credentials before you share your notebook.
def setHadoopConfig64e9c4bd3e4148978db0a312dfcc0a93(name: String) = {
    // This function sets the Hadoop configuration so it is possible to
    // access data from Bluemix Object Storage using Spark

    val prefix = "fs.swift.service." + name
    sc.hadoopConfiguration.set(prefix + ".auth.url", "https://identity.open.softlayer.com" + "/v3/auth/tokens")
    sc.hadoopConfiguration.set(prefix + ".auth.endpoint.prefix","endpoints")
    sc.hadoopConfiguration.set(prefix + ".tenant", "419cd8dece644c82af5a615b62af38e1")
    sc.hadoopConfiguration.set(prefix + ".username", "7e4015cd862d4253b17bfe1c6489c558")
    sc.hadoopConfiguration.set(prefix + ".password", "uYmI.Kj}dWN4e3&G")
    sc.hadoopConfiguration.setInt(prefix + ".http.port", 8080)
    sc.hadoopConfiguration.set(prefix + ".region", "dallas")
    sc.hadoopConfiguration.setBoolean(prefix + ".public", false)
}

// you can choose any name
val name = "keystone"
setHadoopConfig64e9c4bd3e4148978db0a312dfcc0a93(name)

val sqlContext = new org.apache.spark.sql.SQLContext(sc)

val dfData1 = sqlContext.
    read.format("com.databricks.spark.csv").
    option("header", "true").
    option("inferSchema", "true").
    load("swift://IBM411." + name + "/Churn_Rate.csv")
dfData1.show(5)


+----+-------+------------+----------+
|Year|Quarter|Quarter_Year|Churn_Rate|
+----+-------+------------+----------+
|2014|      1|        1Q14|      18.1|
|2014|      2|        2Q14|      18.7|
|2014|      3|        3Q14|      19.3|
|2014|      4|        4Q14|      19.9|
|2015|      1|        1Q15|      20.5|
+----+-------+------------+----------+
only showing top 5 rows



## 1.2 以年份为横坐标，流失率为纵坐标画图

In [5]:
%%brunel data('dfData1') x(QUARTER_YEAR) y(CHURN_RATE) bar tooltip(#all) sort(YEAR:ascending, QUARTER:ascending) tooltip(#all) axes(x:'Time', y:'Churn Rate':grid) 
:: width=800, height=500

## 1.3 导入详细客户信息数据

In [ ]:
//此部分同样需要从Object Storage中读取数据
//1.添加CUST_SUM.csv到本工程的Object Storage
//2.在notebook右上角点击数据窗口，再点击下拉菜单的Insert Spark SQL DataFrame
//3.运行输出数据dfData2

In [3]:

val dfData2 = sqlContext.
    read.format("com.databricks.spark.csv").
    option("header", "true").
    option("inferSchema", "true").
    load("swift://IBM411." + name + "/CUST_SUM.csv")
dfData2.show(5)


+----------+---+---+---------+----------+-------+--------+-----+--------+------------+-------+----------+---------+-----+----------------+---------+-----------+
|   CUST_ID|SEX|AGE|EDUCATION|INVESTMENT| INCOME|ACTIVITY|CHURN|YRLY_AMT|AVG_DAILY_TX|YRLY_TX|AVG_TX_AMT|NEGTWEETS|STATE| EDUCATION_GROUP|TwitterID|CHURN_LABEL|
+----------+---+---+---------+----------+-------+--------+-----+--------+------------+-------+----------+---------+-----+----------------+---------+-----------+
|1009530860|  F| 84|        2|    114368|3852862|       5|    0|700259.0|    0.917808|    335|   2090.32|        3|   TX|Bachelors degree|        0|      false|
|1009544000|  F| 44|        2|     90298|3849843|       1|    0|726977.0|    0.950685|    347|   2095.04|        2|   CA|Bachelors degree|        0|      false|
|1009534260|  F| 23|        2|     94881|3217364|       1|    1|579084.0|    0.920548|    336|   1723.46|        5|   CA|Bachelors degree|        0|       true|
|1009574010|  F| 24|        2|    

## 1.4 以客户所在地区为横坐标，收入为纵坐标画图

In [6]:
%%brunel data('dfData2') x(STATE) y(INCOME) mean(INCOME) bar tooltip(#all) sort(INCOME:ascending) tooltip(#all) axes(x:'State', y:'Income':grid) 
:: width=800, height=500

## 1.5 以客户收入为标记画地图

In [7]:
%%brunel data('dfData2') map key(STATE) x(STATE) color(INCOME) mean(INCOME) label(STATE) tooltip(#all) :: width=800, height=500

## 1.6 分别以客户年龄、平均交易额，平均交易量和收入为横坐标，以是否流失为纵坐标画图

In [8]:

%%brunel data('dfData2') 
         x(AGE) y(#count:linear) color(CHURN_LABEL) bin(AGE) interaction(select) stack bar tooltip(#all) filter(CHURN_LABEL) legends(none) |
         x(AVG_DAILY_TX) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_DAILY_TX) stack bar tooltip(#all) axes(x:10:'AVG DAILY TX', y)|
         x(AVG_TX_AMT) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_TX_AMT) stack bar tooltip(#all)  axes(x,y) legends(none) |
        x(INCOME) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(INCOME) stack bar tooltip(#all) tooltip(#all) axes(x,y) legends(none)
:: width=800, height=600

## 1.7 分别以客户性别、社区活动和教育程度为横坐标，以是否流失为纵坐标画图

In [9]:
%%brunel data('dfData2')
         x(SEX) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(SEX) interaction(select) filter(CHURN_LABEL) axes(x:'GENDER', y) legends(none) |
         x(ACTIVITY) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(ACTIVITY) opacity(#selection)  | 
         x(EDUCATION_GROUP) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(#count) opacity(#selection)  axes(x:'',y) legends(none)
:: width=900, height=600

## 1.8 以客户是否流失为标记画地图

In [10]:
%%brunel data('dfData2') map key(STATE) x(STATE) color(CHURN) mean(CHURN) label(STATE) tooltip(#all) :: width=800, height=500

# 2. 客户流失分类器模型训练
## 2.1 导入相应的库

In [11]:
//import libraries
import org.apache.spark.ml.feature.{StringIndexer,VectorAssembler}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.tuning.{CrossValidator,ParamGridBuilder}

 ## 2.2 把数据切分为训练和测试数据

In [12]:
val Array(training, test) = dfData2.randomSplit(Array(0.8, 0.2), seed = 11L)
println("The number of training data is ",training.count())
println("The number of test data is ",test.count())

(The number of training data is ,4884)
(The number of test data is ,1117)


## 2.3 把string类型的数据转为int型，并选取相应的列组成特征向量

In [13]:
//Feature definition
val genderIndexer = new StringIndexer().setInputCol("SEX").setOutputCol("gender_code")
val stateIndexer = new StringIndexer().setInputCol("STATE").setOutputCol("state_code")
val labelIndexer = new StringIndexer().setInputCol("CHURN").setOutputCol("label")

val featuresAssembler = new VectorAssembler().setInputCols(Array("AGE", 
                                                         "ACTIVITY", 
                                                         "EDUCATION", 
                                                         "NEGTWEETS" ,
                                                         "INCOME",
                                                         "gender_code",
                                                         "state_code")).setOutputCol("features")

## 2.4 使用逻辑回归算法，构建Pipeline, 然后进行交叉验证

In [14]:
//Select model automatically in candidate algorithm - Logistic Regression, SVM or Decision Tree?
val lr = new LogisticRegression().setRegParam(0.01).setLabelCol("label").setFeaturesCol("features")
val pipeline =new Pipeline().setStages(Array(labelIndexer, genderIndexer, stateIndexer, featuresAssembler,lr))
val auc_eval = new BinaryClassificationEvaluator()
val grid = new ParamGridBuilder().addGrid(lr.regParam, Array(1e-3, 1e-2)).addGrid(lr.elasticNetParam,Array(0.25, 0.0)).build()
val cross_val = new CrossValidator().setEstimator(pipeline).setEvaluator(auc_eval).setEstimatorParamMaps(grid).setNumFolds(3)

## 2.5 训练模型，对test数据进行预测

In [15]:
val pipeline_model = cross_val.fit(training)
val trainResult=pipeline_model.transform(test)

## 2.6 评价模型，并画出ROC curve

In [16]:
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
val testingResultScores = trainResult.select("prediction","label").rdd.map(r => (r(0).asInstanceOf[Double], r(1) .asInstanceOf[Double]))
val bc = new BinaryClassificationMetrics(testingResultScores)
val roc = bc.roc

In [20]:
val rocDF = spark.createDataFrame(roc).
                    withColumnRenamed("_1", "FPR").
                    withColumnRenamed("_2", "TPR")
rocDF.show()

+--------------------+------------------+
|                 FPR|               TPR|
+--------------------+------------------+
|                 0.0|               0.0|
|0.004813477737665...|0.9545454545454546|
|                 1.0|               1.0|
|                 1.0|               1.0|
+--------------------+------------------+



In [22]:
%%brunel data('rocDF') x(FPR) y(TPR) line tooltip(#all) axes(x:'False Positive Rate':grid, y:'True Positive Rate':grid)

In [21]:
case class Person(AGE:Int,ACTIVITY:Int,EDUCATION:Int,NEGTWEETS:Int,INCOME:Int,SEX:String,STATE:String)
val dataFrame=Seq(Person(40,1,3,4,200000,"M","TX"),Person(40,1,3,8,200000,"M","OR"))
val df = spark.createDataFrame(dataFrame)
df.show()
pipeline_model.transform(df).show()

+---+--------+---------+---------+------+---+-----+
|AGE|ACTIVITY|EDUCATION|NEGTWEETS|INCOME|SEX|STATE|
+---+--------+---------+---------+------+---+-----+
| 40|       1|        3|        4|200000|  M|   TX|
| 40|       1|        3|        8|200000|  M|   OR|
+---+--------+---------+---------+------+---+-----+

+---+--------+---------+---------+------+---+-----+-----------+----------+--------------------+--------------------+--------------------+----------+
|AGE|ACTIVITY|EDUCATION|NEGTWEETS|INCOME|SEX|STATE|gender_code|state_code|            features|       rawPrediction|         probability|prediction|
+---+--------+---------+---------+------+---+-----+-----------+----------+--------------------+--------------------+--------------------+----------+
| 40|       1|        3|        4|200000|  M|   TX|        0.0|      27.0|[40.0,1.0,3.0,4.0...|[4.52523954309364...|[0.98928395807897...|       0.0|
| 40|       1|        3|        8|200000|  M|   OR|        0.0|       5.0|[40.0,1.0,3.0,8.0